In [ ]:
import numpy as np
from helper import log_multi_target_run, transform_targets, compute_metrics
from tiny_mlflow import log_multi_target_run_local
import xgboost as xgb
import gc
import cudf
import cupy as cp

In [ ]:
DROP = ['id']
TARGETS = ["n2o", "no3", "yield", "soc"]

gdf = cudf.read_parquet("../data_processing/data/df_WIWH_training.parquet")

gdf = gdf[:1000]

X_train = gdf.drop(columns=TARGETS)
y_train = gdf[TARGETS]

del gdf
gc.collect()


0

In [11]:
import time
import os
import json
import numpy as np
from sklearn.model_selection import KFold, GroupKFold



def _to_serializable(obj):
    """Recursively convert numpy types to plain Python types for JSON."""
    if isinstance(obj, (np.floating, np.float32, np.float64)):
        return float(obj)
    if isinstance(obj, (np.integer, np.int32, np.int64)):
        return int(obj)
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, dict):
        return {k: _to_serializable(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [_to_serializable(v) for v in obj]
    return obj



# assumes:
# - TARGETS is defined
# - transform_targets, compute_metrics, log_multi_target_run,
#   log_multi_target_run_local, and xgboost (as xgb) are imported


def train_xgb_models_cv(
    X_cudf,
    y_cudf,
    n_splits=5,
    group_col="id",          # used only for splitting, not as feature
    random_state=42,
    shuffle=True,
    num_boost_round=10_000,
    early_stopping_rounds=200,
    verbose_eval=False,
    log_to_mlflow=False,
    base_dir="experiments_cv",
    experiment_name="multi_target_xgb_cv",
):
    """
    Cross validate XGBoost models for multiple targets and save everything in experiments_cv.

    Parameters
    ----------
    X_cudf : cuDF DataFrame
        Features plus possibly an id column used only for grouping.
    y_cudf : cuDF DataFrame
        Targets with columns matching TARGETS.
    """

    t_global_start = time.time()
    os.makedirs(base_dir, exist_ok=True)

    n_samples = len(X_cudf)
    indices = np.arange(n_samples)

    # 1 - handle grouping column
    groups = None
    feature_cols = list(X_cudf.columns)

    if group_col is not None:
        if group_col in feature_cols:
            # take groups from this column
            groups = X_cudf[group_col].to_pandas().values
            # remove id from feature columns so it is not used as a feature
            feature_cols = [c for c in feature_cols if c != group_col]
            print(f"[CV] Using '{group_col}' as group column and dropping it from features")
        else:
            # group_col specified but not in X_cudf
            # in that case we fall back to plain KFold
            print(
                f"[CV] Warning: group_col='{group_col}' not in X_cudf columns. "
                "Using standard KFold without grouping."
            )
            group_col = None

    # 2 - choose splitter
    if group_col is not None and groups is not None:
        print(f"[CV] Using GroupKFold over group_col='{group_col}'")
        splitter = GroupKFold(n_splits=n_splits)
        split_iter = splitter.split(indices, groups=groups)
        split_name = f"GroupKFold(n_splits={n_splits}, group_col='{group_col}')"
    else:
        print("[CV] Using standard KFold")
        splitter = KFold(
            n_splits=n_splits,
            shuffle=shuffle,
            random_state=random_state,
        )
        split_iter = splitter.split(indices)
        split_name = (
            f"KFold(n_splits={n_splits}, shuffle={shuffle}, random_state={random_state})"
        )

    print(f"[CV] Starting XGBoost cross validation")
    print(f"[CV] Splitter: {split_name}")
    print(f"[CV] Total samples: {n_samples}")
    print(f"[CV] Targets: {TARGETS}")
    print("=" * 80)

    # 3 - target transforms
    print("[CV] Fitting target transformations on full dataset ...")
    t_tr_start = time.time()
    y_orig_np, y_trans_np, transformers = transform_targets(y_cudf)
    print(f"[CV] Target transformations done in {time.time() - t_tr_start:.1f}s")

    # 4 - build feature matrix on CPU without id
    print("[CV] Moving features to CPU (numpy float32) ...")
    t_x_start = time.time()

    # only use feature_cols, which does not contain group_col/id
    X_features_cudf = X_cudf[feature_cols]
    X_all_np = X_features_cudf.to_numpy().astype("float32")
    feature_names = feature_cols

    print(
        f"[CV] X feature shape (without group_col): {X_all_np.shape}, "
        f"converted in {time.time() - t_x_start:.1f}s"
    )
    print(f"[CV] Feature columns used: {feature_names}")
    print("=" * 80)

    # 5 - base xgb params
    base_params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "hist",
        "device": "cuda",
        "sampling_method": "gradient_based",
        "max_bin": 256,
        "max_depth": 9,
        "min_child_weight": 48,
        "subsample": 0.8,
        "colsample_bytree": 0.6,
        "learning_rate": 0.013556,
        "gamma": 1.074052,
        "reg_alpha": 0.151021,
        "reg_lambda": 11.097351,
    }

    fold_models = {target: [] for target in TARGETS}
    cv_results = {target: {"fold_metrics": []} for target in TARGETS}
    splits_info = []

    # 6 - CV loop
    for fold_idx, (train_idx, val_idx) in enumerate(split_iter, start=1):
        t_fold_start = time.time()
        n_train = len(train_idx)
        n_val = len(val_idx)

        print("")
        print("=" * 80)
        print(
            f"[CV] Fold {fold_idx}/{n_splits} - "
            f"train samples: {n_train}, val samples: {n_val}"
        )
        print("=" * 80)

        splits_info.append(
            {
                "fold": fold_idx,
                "n_train": int(n_train),
                "n_val": int(n_val),
            }
        )

        X_train_np = X_all_np[train_idx]
        X_val_np = X_all_np[val_idx]

        models_fold = {}
        metrics_fold = {}

        for target in TARGETS:
            t_target_start = time.time()
            print(f"[CV][Fold {fold_idx}] Target '{target}' - preparing data ...")

            y_train_trans = y_trans_np[target][train_idx].astype("float32")
            y_val_trans = y_trans_np[target][val_idx].astype("float32")
            y_val_orig = y_orig_np[target][val_idx]

            dtrain = xgb.DMatrix(
                X_train_np,
                label=y_train_trans,
                feature_names=feature_names,
            )
            dval = xgb.DMatrix(
                X_val_np,
                label=y_val_trans,
                feature_names=feature_names,
            )

            print(f"[CV][Fold {fold_idx}] Target '{target}' - training XGBoost ...")

            params = dict(base_params)
            model_name = f"xgb_v1_cv_fold{fold_idx}_{target}"

            evals = [(dtrain, "train"), (dval, "valid")]

            booster = xgb.train(
                params=params,
                dtrain=dtrain,
                num_boost_round=num_boost_round,
                evals=evals,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=verbose_eval,
            )

            fold_models[target].append(booster)
            models_fold[target] = booster

            preds_val_trans = booster.predict(dval)

            fold_metrics, _ = compute_metrics(
                target,
                preds_val_trans,
                y_val_orig,
                transformers[target],
                model_name,
            )
            cv_results[target]["fold_metrics"].append(
                {"fold": fold_idx, **fold_metrics}
            )
            metrics_fold[target] = fold_metrics

            print(
                f"[CV][Fold {fold_idx}] Target '{target}' done in "
                f"{time.time() - t_target_start:.1f}s "
                f"| val RMSE: {fold_metrics.get('rmse', float('nan')):.4f} "
                f"MAE: {fold_metrics.get('mae', float('nan')):.4f} "
                f"R2: {fold_metrics.get('r2', float('nan')):.4f}"
            )

            # save model for this fold and target
            model_dir = os.path.join(
                base_dir, experiment_name, f"fold_{fold_idx}", "models"
            )
            os.makedirs(model_dir, exist_ok=True)
            model_path = os.path.join(model_dir, f"{target}.json")
            booster.save_model(model_path)

        # per fold logging
        run_name = f"xgb_cv_fold{fold_idx}"
        if log_to_mlflow:
            log_multi_target_run(
                model_family_name="xgboost",
                models=models_fold,
                metrics=metrics_fold,
                feature_names=feature_names,
                transformers=transformers,
                experiment_name=experiment_name,
                run_name=run_name,
            )
        else:
            log_multi_target_run_local(
                model_family_name="xgboost",
                models=models_fold,
                metrics=metrics_fold,
                feature_names=feature_names,
                transformers=transformers,
                experiment_name=experiment_name,
                run_name=run_name,
                base_dir=base_dir,
            )

        print(
            f"[CV] Completed fold {fold_idx}/{n_splits} in "
            f"{time.time() - t_fold_start:.1f}s"
        )

    # 7 - aggregate metrics
    print("")
    print("=" * 80)
    print("[CV] Aggregating metrics across folds")
    print("=" * 80)

    for target in TARGETS:
        rmse_list = [m["rmse"] for m in cv_results[target]["fold_metrics"]]
        mae_list = [m["mae"] for m in cv_results[target]["fold_metrics"]]
        r2_list = [m["r2"] for m in cv_results[target]["fold_metrics"]]

        mean_rmse = float(np.mean(rmse_list))
        std_rmse = float(np.std(rmse_list))
        mean_mae = float(np.mean(mae_list))
        std_mae = float(np.std(mae_list))
        mean_r2 = float(np.mean(r2_list))
        std_r2 = float(np.std(r2_list))

        cv_results[target]["mean_metrics"] = {
            "rmse": mean_rmse,
            "mae": mean_mae,
            "r2": mean_r2,
        }
        cv_results[target]["std_metrics"] = {
            "rmse": std_rmse,
            "mae": std_mae,
            "r2": std_r2,
        }

        print(
            f"[CV][{target}] "
            f"RMSE: {mean_rmse:.4f} ± {std_rmse:.4f} | "
            f"MAE: {mean_mae:.4f} ± {std_mae:.4f} | "
            f"R2: {mean_r2:.4f} ± {std_r2:.4f}"
        )

    # 8 - save summary
    summary_dir = os.path.join(base_dir, experiment_name, "summary")
    os.makedirs(summary_dir, exist_ok=True)

    summary_payload = {
        "n_splits": n_splits,
        "splitter": split_name,
        "targets": TARGETS,
        "splits_info": splits_info,
        "cv_results": cv_results,
    }

    summary_path = os.path.join(summary_dir, "cv_results.json")
    summary_payload_serializable = _to_serializable(summary_payload)
    with open(summary_path, "w") as f:
        json.dump(summary_payload_serializable, f, indent=2)

    print(f"[CV] Saved CV summary to {summary_path}")
    print("")
    print(
        f"[CV] All folds completed in {time.time() - t_global_start:.1f}s "
        f"for {n_splits} folds and targets: {TARGETS}"
    )
    print("=" * 80)

    return fold_models, cv_results, transformers


In [12]:
%%time

fold_models, cv_results, transformers = train_xgb_models_cv(
    X_train,
    y_train,
    n_splits=5,
    group_col="id",          # used only for splitting, not as feature
    random_state=42,
    shuffle=True,
    num_boost_round=10_000,
    early_stopping_rounds=200,
    verbose_eval=False,
    log_to_mlflow=False,
    base_dir="experiments_cv",
    experiment_name="multi_target_xgb_cv",
)

[CV] Using 'id' as group column and dropping it from features
[CV] Using GroupKFold over group_col='id'
[CV] Starting XGBoost cross validation
[CV] Splitter: GroupKFold(n_splits=5, group_col='id')
[CV] Total samples: 1000
[CV] Targets: ['n2o', 'no3', 'yield', 'soc']
[CV] Fitting target transformations on full dataset ...
[CV] Target transformations done in 0.0s
[CV] Moving features to CPU (numpy float32) ...
[CV] X feature shape (without group_col): (1000, 62), converted in 0.0s
[CV] Feature columns used: ['soil', 'climate', 'cropping_systems', 'crop_rotation', 'n_synth_type', 'n_org_type', 'n_org_replication', 'n_synth_replication', 'irrigation', 'manu_depth', 'n_org_amount', 'n_synthamount', 'fert_amount_1', 'fert_amount_2', 'fert_amount_3', 'manu_amount_1', 'manu_amount_2', 'manu_amount_3', 'prec_days', 'total_nitrogen', 'total_precipitation_year', 'total_average_temperature_year', 'total_precipitation_growing_season', 'total_average_temperature_growing_season', 'total_precipitation

In [11]:
import json
import pandas as pd

path="experiments_cv/multi_target_xgb_cv/experiments.jsonl"
rows=[]
with open(path, 'r') as f:
    for line in f:
        rec=json.loads(line)
        run_name=rec.get("run_name")
        fold=run_name.split("_")[-1] if "_" in run_name else run_name
        fold = int(fold.replace("fold", ""))
        metrics=rec.get("metrics",{})
        for target, m in metrics.items():
            rows.append({
                "target": target,
                "fold": fold,
                "rmse": m.get("rmse"),
                "mae": m.get("mae"),
                "r2": m.get("r2"),
            })
df=pd.DataFrame(rows)

# compute oof per target
oof_list=[]
for target, grp in df.groupby("target"):
    oof_list.append({
        "target": target,
        "fold": "OOF",
        "rmse": grp["rmse"].mean(),
        "mae": grp["mae"].mean(),
        "r2": grp["r2"].mean(),
    })
df_oof=pd.DataFrame(oof_list)

final_df=pd.concat([df, df_oof], ignore_index=True)
final_df.sort_values(['target','fold'] , inplace = True)
final_df = final_df.reset_index(drop = True)

In [13]:
final_df.to_csv('experiments_cv/experiments_cv.csv', index = False)